## Performance profiler

Test 1D minimization -> eigenproblem code

In [ ]:
from Hubbard.equalizer import *
import numpy as np
from scipy.optimize import minimize, shgo, dual_annealing

N = 20
R0 = np.array([3, 3, 7.2])

W = HubbardParamEqualizer(N,
                          R0=R0,
                          lattice=np.array([6], dtype=int),
                          band=1,
                          dim=1,
                          avg=1 / 2,
                          sparse=True,
                          equalize=False,
                          symmetry=True,
                          verbosity=0)

E, V, parity = eigen_basis(W)
R = locality_mat(W, V[0], parity[0])
np.set_printoptions(precision=4, suppress=True)


def f(R):
    X, solution = la.eigh(R)
    U = solution[:, np.argsort(X)]
    return U


def g(R):
    solution = riemann_optimize(W, None, [torch.from_numpy(R)])
    U = site_order(W,  V[0], solution, parity[0])
    return U


%timeit f(R[0])
%timeit g(R[0])
U1 = fix_phase(f(R[0]))
U2 = fix_phase(g(R[0]))
print(U1-U2)


24.9 µs ± 5.78 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
25.6 ms ± 697 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[[ 0.  0.  0.  0. -0.  0.]
 [-0. -0.  0.  0. -0.  0.]
 [ 0.  0.  0.  0. -0.  0.]
 [-0. -0.  0.  0. -0.  0.]
 [-0.  0.  0.  0. -0. -0.]
 [-0.  0. -0. -0.  0.  0.]]


3D

In [ ]:
from Hubbard.equalizer import *
import numpy as np
from scipy.optimize import minimize, shgo, dual_annealing

N = 20
R0 = np.array([3, 3, 7.2])

W = HubbardParamEqualizer(N,
                          R0=R0,
                          lattice=np.array([4], dtype=int),
                          band=1,
                          dim=3,
                          avg=1 / 2,
                          sparse=True,
                          equalize=False,
                          symmetry=True,
                          verbosity=0)

E, V, parity = eigen_basis(W)
R = locality_mat(W, V[0], parity[0])
np.set_printoptions(precision=4, suppress=True)


def f(R):
    X, solution = la.eigh(R)
    U = solution[:, np.argsort(X)]
    return U


def g(R):
    solution = riemann_optimize(W, None, [torch.from_numpy(R)])
    U = site_order(W,  V[0], solution, parity[0])
    return U


%timeit f(R[0])
%timeit g(R[0])
U1 = fix_phase(f(R[0]))
U2 = fix_phase(g(R[0]))
print(U1-U2)


Check if the error of commutativity is from sparse diagonalization

In [ ]:
from Hubbard.equalizer import *
import numpy as np
from scipy.optimize import minimize, shgo, dual_annealing

N = 20
R0 = np.array([3, 3, 7.2])

W = HubbardParamEqualizer(N,
                          R0=R0,
                          lattice=np.array([3, 3], dtype=int),
                          band=1,
                          dim=2,
                          avg=1 / 2,
                          sparse=False,
                          equalize=False,
                          symmetry=True,
                          verbosity=0)

E, V, parity = eigen_basis(W)
R = locality_mat(W, V[0], parity[0])
np.set_printoptions(precision=4, suppress=True)

X, solution = la.eigh(R[0])
solution = riemann_optimize(W, None, [torch.from_numpy(R[0])])

X = solution.conj().T @ R[0] @ solution
Y = solution.conj().T @ R[1] @ solution
print('Full matrix:')
print(la.norm(X - np.diag(np.diag(X))))
print(la.norm(Y - np.diag(np.diag(Y))))


W = HubbardParamEqualizer(N,
                          R0=R0,
                          lattice=np.array([3, 3], dtype=int),
                          band=1,
                          dim=2,
                          avg=1 / 2,
                          sparse=True,
                          equalize=False,
                          symmetry=True,
                          verbosity=0)

E, V, parity = eigen_basis(W)
R = locality_mat(W, V[0], parity[0])
np.set_printoptions(precision=4, suppress=True)

X, solution = la.eigh(R[0])
solution = riemann_optimize(W, None, [torch.from_numpy(R[0])])

X = solution.conj().T @ R[0] @ solution
Y = solution.conj().T @ R[1] @ solution
print('Sparse matrix:')
print(la.norm(X - np.diag(np.diag(X))))
print(la.norm(Y - np.diag(np.diag(Y))))

Full matrix:
0.0001670946088647852
2.9726135583066355
Sparse matrix:
0.00031333472050198914
3.5329038804414434


Profile `site_order()` function

In [ ]:
from Hubbard.equalizer import *
import numpy as np
from scipy.optimize import minimize, shgo, dual_annealing

N = 20
R0 = np.array([3, 3, 7.2])

W = HubbardParamEqualizer(N,
                          R0=R0,
                          lattice=np.array([4], dtype=int),
                          band=1,
                          dim=2,
                          avg=1 / 2,
                          sparse=True,
                          equalize=False,
                          symmetry=True,
                          verbosity=0)

E, V, parity = eigen_basis(W)
R = locality_mat(W, V[0], parity[0])
np.set_printoptions(precision=4, suppress=True)

X, solution = la.eigh(R[0])
%timeit site_order(W, V[0], solution, parity[0])
%timeit np.argsort(X)

Triangular lattice size adjust to: [4 1]
3.89 ms ± 471 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
828 ns ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
from Hubbard.equalizer import *
import numpy as np
from scipy.optimize import minimize, shgo, dual_annealing

N = 20
R0 = np.array([3, 3, 7.2])

W = HubbardParamEqualizer(N,
                          R0=R0,
                          lattice=np.array([3, 3], dtype=int),
                          band=1,
                          dim=2,
                          avg=1 / 2,
                          sparse=True,
                          equalize=False,
                          symmetry=True,
                          verbosity=0)

E, V, parity = eigen_basis(W)
R = locality_mat(W, V[0], parity[0])
np.set_printoptions(precision=4, suppress=True)

# X, solution = la.eigh(R[0])
solution = riemann_optimize(W, None, [torch.from_numpy(R[0])])

X = solution.conj().T @ R[0] @ solution
Y = solution.conj().T @ R[1] @ solution
print(np.diag(X))
Yp, Yeig = la.eigh(Y)
print(Yp)

shift = np.zeros((W.Nsite, 2))
for i in range(W.Nsite):
    shift[i, :2] = W.trap_centers[i] * W.lc
x = np.array([shift[i, :] for i in range(W.Nsite)])
print(x)

def so1(dvr: MLWF, W, U, p):
    # Order Wannier functions by lattice site label
    shift = np.zeros((dvr.Nsite, dim))
    for i in range(dvr.Nsite):
        shift[i, :2] = dvr.trap_centers[i] * dvr.lc
        # Small offset to avoid zero values on z=0 in odd sector
        shift[i, 2] = 0.25
    x = [[[shift[i, d]] for d in range(dim)] for i in range(dvr.Nsite)]
    center_val = np.zeros((dvr.Nsite, dvr.Nsite))
    for i in range(dvr.Nsite):
        center_val[i, :] = abs(wannier_func(dvr, W, U, p, x[i]))
    # print(center_val)
    # Find at which trap max of Wannier func is
    order = np.argmax(center_val, axis=1)
    if dvr.verbosity > 1:
        print("Trap site position of Wannier functions:", order)
        print("Order of Wannier functions is set to match traps.")
    return order

o1 = so1(W, V[0], solution, parity[0])
print(o1)

Triangular lattice size adjust to: [3 3]
[-0.      0.      1.4663 -1.4663  1.4663  0.     -1.4664  1.4665 -1.4662]
[-1.6698 -1.6694 -1.6693 -0.      0.      0.      1.6693  1.6694  1.6698]
[[-1.52 -1.69]
 [-1.52  0.  ]
 [-1.52  1.69]
 [ 0.   -1.69]
 [ 0.    0.  ]
 [ 0.    1.69]
 [ 1.52 -1.69]
 [ 1.52  0.  ]
 [ 1.52  1.69]]
[8 3 6 0 1 5 4 2 7]


Given the numericall error, we are not able to locate which Wannier function in which site by simply calculate $W^*XW$, $W^*YW$.

Profile equalization

In [ ]:
from Hubbard.plot import *
import numpy as np
from scipy.optimize import minimize, shgo, dual_annealing

N = 20
R0 = np.array([3, 3, 7.2])

G = HubbardGraph(N,
                 R0=R0,
                 lattice=np.array([4], dtype=int),
                 band=1,
                 dim=2,
                 avg=1 / 2,
                 sparse=True,
                 equalize=False,
                 symmetry=True,
                 verbosity=0)

G.eq_label = 'eq'
%prun __, __, info = G.equalzie('Uvt', callback=True)
print(f'V_err = {np.std(np.diag(G.A)) / abs(np.mean(np.diag(G.A)))}')
t = abs(G.nn_tunneling(G.A))
print(f't_err = {np.std(t) / np.mean(t)}')
print(f'U_err = {np.std(G.U) / np.mean(G.U)}')


i=20	c_i=0.11212180717376728	c_i//2-c_i=0.17076978692250716
i=40	c_i=0.14040197359367773	c_i//2-c_i=-0.026662759579776527
i=60	c_i=0.08451466763754942	c_i//2-c_i=-0.0005978690416478111
i=80	c_i=0.07556727520564425	c_i//2-c_i=0.003938816843545509
i=100	c_i=0.07321163129474742	c_i//2-c_i=0.0023105214837223637
i=120	c_i=0.07255777549453328	c_i//2-c_i=0.0014423590529585861
i=140	c_i=0.07243974408924818	c_i//2-c_i=0.0007282871138456909
i=160	c_i=0.07241181384007411	c_i//2-c_i=0.0003743584375251324
i=180	c_i=0.07240388001095949	c_i//2-c_i=0.00019904571820680839
i=200	c_i=0.07239852262748916	c_i//2-c_i=9.922372212978203e-05
i=220	c_i=0.07239871817384697	c_i//2-c_i=4.980428542464277e-05
i=240	c_i=0.07239871699744573	c_i//2-c_i=2.393478842047103e-05
 V_err = 0.010358357920899283
t_err = 1.4100296960767573e-07
U_err = 0.025847775927205552


         6430887 function calls (6342930 primitive calls) in 34.238 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   111348   14.800    0.000   19.923    0.000 arpack.py:533(iterate)
404375/373656   12.105    0.000   15.374    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   335049    2.597    0.000    2.597    0.000 {built-in method numpy.core._multiarray_umath.c_einsum}
   111348    1.393    0.000    4.754    0.000 core.py:374(H_op)
    12048    0.239    0.000    0.287    0.000 function_base.py:3537(sinc)
   362411    0.236    0.000    0.236    0.000 {method 'reshape' of 'numpy.ndarray' objects}
      502    0.229    0.000   13.360    0.027 core.py:573(wannier_func)
   111348    0.201    0.000    5.146    0.000 _interface.py:201(matvec)
      502    0.183    0.000    0.185    0.000 arpack.py:573(extract)
     2008    0.160    0.000    0.473    0.000 core.py:295(delta)
     2008    0.131  